## Tutorial: 

### Configure the Environment

In [ ]:
import json
import os
import chi,chi.ssh
from datetime import datetime, timedelta
from dateutil import tz

import chi

#Config with your project and site
chi.use_site('CHI@UC')
chi.set('project_name', 'CHI-231095')

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
prefix = username + "_k8s_ml_"
server_name = prefix + 'Server'
network_name = prefix + 'Net'
subnet_name = prefix + 'Subnet'
router_name = prefix + 'Router'
lease_name = prefix + 'Lease'

#Server attributes
image_name='CC-Ubuntu20.04'
node_type="compute_cascadelake_r"
server_count=3
physical_network="physnet1"
subnet_cidr = '192.168.1.0/24'


Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


### Isolated Network

#### Create the Network

In [ ]:
import chi.network

# Create a network out of provider network physnet1
network = chi.network.create_network(network_name, provider='physnet1')
network_id = network['id']

print(f'Network ID: {network_id}')

Network ID: fb495a06-ff0b-41f9-ac23-54c47a264991


#### Configure the Network


In [ ]:
subnet = chi.network.create_subnet(subnet_name, network_id, cidr=subnet_cidr, gateway_ip=None)
router = chi.network.create_router(router_name, gw_network_name='public')
chi.network.add_subnet_to_router(router['id'], subnet['id'])

{'id': '7b1c0e0f-e14b-4715-bad8-4d3a5cd23118',
 'tenant_id': 'cb970a4b0f2e42c9b1b3f9015d02f8a5',
 'port_id': 'deececf9-a233-4c0a-9f4c-f071f791720d',
 'network_id': 'fb495a06-ff0b-41f9-ac23-54c47a264991',
 'subnet_id': '0d82ffe0-67a3-4e41-91c4-76936dc42cfd',
 'subnet_ids': ['0d82ffe0-67a3-4e41-91c4-76936dc42cfd']}

### Servers on the Network



#### Create a Lease

In [ ]:
import chi.lease

BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'

# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=7)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
chi.lease.add_node_reservation(reservation_list, count=server_count, node_type=node_type)
chi.lease.add_fip_reservation(reservation_list, count=server_count)

# Create the lease
lease = chi.lease.create_lease(lease_name, 
                               start_date=start_date,
                               end_date=end_date,
                               reservations=reservation_list)

lease_id = lease["id"]

chi.lease.wait_for_active(lease_id)

{'created_at': '2023-04-29 18:22:47',
 'updated_at': '2023-04-29 18:23:09',
 'id': 'f13fceb5-b470-4430-8571-b8959d917f33',
 'name': 'cp3793_nyu_edu_k8s_ml_Lease',
 'user_id': '50de6f77f6d1941774ecf322c9f0ad2a3e2c128f4707c3278d98fd4a98b86d85',
 'project_id': 'cb970a4b0f2e42c9b1b3f9015d02f8a5',
 'start_date': '2023-04-29T18:23:00.000000',
 'end_date': '2023-05-06T18:22:00.000000',
 'trust_id': 'e930d5ae9ae4497c974adbd43af8adbb',
 'status': 'ACTIVE',
 'degraded': False,
 'reservations': [{'created_at': '2023-04-29 18:22:48',
   'updated_at': '2023-04-29 18:23:06',
   'id': '7971a349-c1f5-404b-941f-b7d02003fb0d',
   'lease_id': 'f13fceb5-b470-4430-8571-b8959d917f33',
   'resource_id': '5ed7beff-d244-48c2-929c-da5781a5c61f',
   'resource_type': 'virtual:floatingip',
   'status': 'active',
   'missing_resources': False,
   'resources_changed': False,
   'network_id': '44b38c44-2a42-4b6d-b129-6c8f1b2a1375',
   'amount': 3,
   'required_floatingips': []},
  {'created_at': '2023-04-29 18:22:47'

#### Get the Reservations

In [ ]:
compute_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'physical:host'][0]['id']
floatingip_reservation_id = [reservation for reservation in lease['reservations'] if reservation['resource_type'] == 'virtual:floatingip'][0]['id']

print(f"compute_reservation_id: {compute_reservation_id}")
print(f"floatingip_reservation_id: {floatingip_reservation_id}")

compute_reservation_id: a32bca2b-8486-49ef-a91a-22095092b0ef
floatingip_reservation_id: 7971a349-c1f5-404b-941f-b7d02003fb0d


#### Start the Server

The `create_server` method won't work until your lease is active! Verify your lease is active by going to __Reservations__ > __Leases__ in your Chameleon project, and checking that the lease with your name has the status _ACTIVE_.

Don't worry if you ran it too early and your server (in __Compute__ > __Instances__) has the _Error_ status -- if this happens, you can just delete your instance using the Chameleon interface and run this cell again.

In [ ]:
#create the server
server = chi.server.create_server(server_name, 
                                  compute_reservation_id,
                                  network_id=network_id,
                                  nics=[],
                                  image_name=image_name,
                                  count=3)


#### Associate the Floating IP


In [ ]:
nodes = {}

for j,i in enumerate(server):
    chi.server.wait_for_active(i.id)
    floating_ip = chi.server.associate_floating_ip(i.id)
    nodes[j] = floating_ip

    print(f'Floating IP of node_{j}: {floating_ip}')

Floating IP of node_0: 192.5.86.235
Floating IP of node_1: 192.5.87.228
Floating IP of node_2: 192.5.87.183


#### Generate ssh key for each host

In [ ]:
import chi, chi.ssh
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)
    remote = chi.ssh.Remote(node_ip)
    remote.run('ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -q -N ""')

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.86.235: b'd417e5f99e05882001a05dd0d5013ea5'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.87.228: b'e2951c7952be44fb67a38199ec2ad2ed'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/paramiko/client.py:835: UserWarning: Unknown ssh-ed25519 host key for 192.5.87.183: b'274d1ee82d96139a266845eebc76c6cd'
  warnings.warn(


#### Key Transfer
Add each hosts public key to the authorized_keys file located in ~/.ssh folder in every host to make sure that all the hosts are able to ssh into each other.

In [ ]:
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)
    remote = chi.ssh.Remote(node_ip)
    public_key = remote.run('cat ~/.ssh/id_rsa.pub').tail("stdout")[2:]
    public_key = public_key.replace("\n", "")
    for node_ip in nodes.values():
        remote = chi.ssh.Remote(node_ip)
        remote.run(f'echo {public_key} >> ~/.ssh/authorized_keys') 

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQC7Xn1CXX6/EeFrjw38+WzJ+JBVZhuq1R3lhaNt7uacjax88LO9SVtZPo7vBZgjRN4N7IHjOgWhJnuZR4C3sLWyTnJqT6r4S45rQdBquDdmmJJY/K73kLrN+R3bwpMrM1kXbVgowv6ZSS5vHItZPQXH6gKsJEiSCv+gU2cajtLRxqMwmT5vYpbIpul8iXsn1O+JWrz0h9wwtiuThe9aQ/piMY0gJfkWSHinE+9DM1/NaITxQ4qzVvnSa/pW4bURb7Do8CZ0BjUwnJnK9mLJniqdAinxGupxekSfD6Rg3bIul28pNCj8Cyeejdv1Y6uxDHWGPkqKpHz42C87N/3Bh9xw3yZ6YEHPKq+yOKiKcY2nlPanKDw/r0N6zLYk6WY1ITE6DKo4iqTEnh1hb5dWZqxhGcP+UUUjO6yqBsBSr5mVxyzPY9e6NZujxcCso2fBfL0iWoyKTeQ16WVFqzViezVbO2mh/6z7PklGkZwNce6cRSi3B91gfhk+FpGC3vW64Euu8GUv9zg5DXES5Pwd5o5bDL5wHAz/NP5tzVYu2jHx5b8vk05OI6PRjzS9iuxFbYLVtKgBLyw4/6Gk+JC+Ue1ZJex4vH1ZWcS7EZrmnFk0Qhhc8+91ibW1GeX5BP8a+HsGO6sIZnPZv1W9iDrxBlwN41X0HplI/iyMQGt2uFPbLQ== cc@cp3793-nyu-edu-k8s-ml-server-3
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQC6IoU6e7/wtokQ9MeLYHM+kYpipB/vCVRlmEVcSYetwb4WC0eCHEoqr9LTlztM1YjZYNU2O5oSLz66T4j8KprgiwftEwwMwGg3orwLJaoqEXUBiK0cIrVMoJZBIksSP2qo47zF2PHYucRAS1Y6s40fpte0WaGXL0rOAP2WX80frLqO9U8yvVRj7EKGdo+kz7uUUUpz0OmLK

#### Store the physical ip of each host
During the process of creating the cluster we will be needing the physical ips of each host so this will be usefull.

In [ ]:
physical_ips=[]
for node_ip in nodes.values():
    chi.server.wait_for_tcp(node_ip, port=22)
    remote = chi.ssh.Remote(node_ip)
    physical_ips.append(remote.run('hostname -I').tail("stdout")[2:-1])

192.168.1.15 
192.168.1.215 
192.168.1.101 


#### setting up the cluster

In [ ]:
ansible_node = nodes[1]
remote = chi.ssh.Remote(ansible_node)
remote.run("git clone https://github.com/kubernetes-sigs/kubespray.git")
remote.run("cd kubespray; git checkout release-2.16")
remote.run("cd kubespray; sudo pip3 install -r requirements.txt")
remote.run("cd kubespray; cp -rfp inventory/sample inventory/mycluster")
remote.run(f"declare -a IPS=({physical_ips[0]} {physical_ips[1]} {physical_ips[2]});"+"cd kubespray; CONFIG_FILE=inventory/mycluster/hosts.yaml python3 contrib/inventory_builder/inventory.py ${IPS[@]}")

Cloning into 'kubespray'...


Branch 'release-2.16' set up to track remote branch 'release-2.16' from 'origin'.


Switched to a new branch 'release-2.16'


  Created wheel for ansible: filename=ansible-2.9.20-py3-none-any.whl size=16203441 sha256=dfd0000f7357960de2477700554841dfa77da82219e1d4aa212d0760b72e2c73
  Stored in directory: /root/.cache/pip/wheels/61/a8/07/9b4017c5cbc3e015ecb6cbc5b37aa974cb485cae9d46bc6728
Successfully built ansible
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 1.1.0
    Not uninstalling markupsafe at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.10.1
    Not uninstalling jinja2 at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'Jinja2'. No files were found to uninstall.
  Attempting uninstall: ansible
    Found existing installation: ansible 2.9.6
    Not uninstalling ansible at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'ansible'. No files were found to uninstall.
  

<Result cmd='cd kubespray; CONFIG_FILE=inventory/mycluster/hosts.yaml python3 contrib/inventory_builder/inventory.py ${IPS[@]}' exited=0>

#### Delete Server

In [ ]:
for i in server:
  chi.server.delete_server(i.id)

#### De-configure Network



In [ ]:
router_id = router['id']
subnet_id = subnet['id']

try:
    result = chi.network.remove_subnet_from_router(router_id, subnet_id)
except Exception as e:
    print(f"detach_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_router(router_id)
except Exception as e:
    print(f"delete_router_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_subnet(subnet_id)
except Exception as e:
    print(f"delete_subnet_by_name error: {str(e)}")
    pass

try:
    result = chi.network.delete_network(network_id)
except Exception as e:
    print(f"delete_network_by_name error: {str(e)}")
    pass


#### Release Lease

In [ ]:
chi.lease.delete_lease(lease_id)